<a href="https://colab.research.google.com/github/lemcat55/w7StatCalc/blob/main/Week7_ML_Predictive_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1: ANOVA prediction

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn import linear_model
from scipy.stats import f
##  Enter your data #####################################################
df=pd.read_csv("http://algorain.com/Courses/Statistics/w6/RealEstate.csv")
dependant='Price'
X = df[['Footage', 'Age']]
point_of_interest=[2500,12.]
########################
#df=pd.read_csv("http://algorain.com/Courses/Statistics/w6/Passengers.csv")
#dependant='Cost'
#X = df[['nPassengers']]
#point_of_interest=[91.]
########################
#df=pd.read_csv("http://algorain.com/Courses/Statistics/w1/Project_Data_Set.csv")
#dependant='Sales (Y)'
#X = df[['Calls (X1)','Time (X2)','Years (X3)']]
#point_of_interest=[160, 17, 3.]
#########################################################################
y1 = df[dependant]
print(df.head())
print('(rows,cols) =',df.shape)
m=6 # display pesicion
k=X.shape[1]
n=X.shape[0]
yBar=y1.mean()
model = linear_model.LinearRegression()
model.fit(X, y1)
b1=model.coef_
b0=model.intercept_
pred_col=model.predict(X)
SSR=((pred_col-yBar)**2).sum()
dy=df[dependant]-pred_col
SSE=(dy**2).sum()
MSE=SSE/(n-k-1)
se=np.sqrt(MSE)
R2=SSR/(SSR+SSE)
R=np.sqrt(R2)
AdjR2=1-SSE*(n-1)/((SSR+SSE)*(n-k-1))
print('\nSUMMARY OUTPUT\n')
print('Regression Statistics')
print('Multiple R:\t', R)
print('R square:\t', R2)
print('Adjasted R:\t', AdjR2)
print('Observations:\t', n)
print('\nANOVA')
MSR=SSR/k
MSE=SSE/(n-k-1)
F=MSR/MSE
p = 1 - f.cdf(F,k,n-k-1)
d={'':['Regression','Residual','Total'],\
   'df':[k,	n-k-1, n-1],\
   'SS':[str(np.round(SSR,m)),str(np.round(SSE,m)),str(np.round(SSE+SSR,m))],\
   'MS':[str(np.round(MSR,m)),str(np.round(MSE,m)),''],\
   'F':[str(np.round(F,m)),'',''],\
   'Significance':[str(np.round(p,2*m)),'','']}
df = pd.DataFrame(d) 
print(df)
s='\nModel: '+dependant+" = "+str(np.round(b0,m))
i=0
for col in X.columns:
    s+=' + ('+str(np.round(b1[i],m))+') * '+col
    i+=1
print(s)
print('\nFor X =', point_of_interest,'  Y =',b0+(b1*point_of_interest).sum())

   Price  Footage  Age
0   63.0     1605   35
1   65.1     2489   45
2   69.9     1553   20
3   76.8     2404   32
4   73.9     1884   25
(rows,cols) = (23, 3)

SUMMARY OUTPUT

Regression Statistics
Multiple R:	 0.8608726813513168
R square:	 0.7411017734970058
Adjasted R:	 0.7152119508467064
Observations:	 23

ANOVA
               df           SS           MS          F  Significance
0  Regression   2  8189.723012  4094.861506  28.625216  1.352978e-06
1    Residual  20  2861.016988   143.050849                         
2       Total  22     11050.74                                      

Model: Price = 57.350746 + (0.017718) * Footage + (-0.666348) * Age

For X = [2500, 12.0]   Y = 93.64966107004997


#2: ML prediction

In [ ]:
# see https://jovian.ai/aakashns/02-linear-regression
# https://www.manning.com/books/deep-learning-with-python#toc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
##  Enter your data #################
df=pd.read_csv("http://algorain.com/Courses/Statistics/w1/Project_Data_Set.csv")
dependant='Sales (Y)'
X = df[['Calls (X1)','Time (X2)','Years (X3)']]
point_of_interest=[160, 17, 3.]
batch_size = 5
learning_rate=1e-5
nEpoch=100
nStep=10
####################
print(df.head())
inputs=df.to_numpy()[:,1:4]
targets=df.to_numpy()[:,0:1]
inputs=np.array(inputs,dtype='float32')
targets=np.array(targets,dtype='float32')
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)#data is converted

train_ds = TensorDataset(inputs, targets)# Define dataset
train_dl = DataLoader(train_ds, batch_size, shuffle=True)# Define data loader
model = nn.Linear(3, 1)# Define model
loss_fn = F.mse_loss# Define loss function
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)# Define optimizer
def fit(num_epochs, model, loss_fn, opt, train_dl):# Define Utility function
  for epoch in range(num_epochs):# Repeat for given number of epochs
    for xb,yb in train_dl:# Train with batches of data
      pred = model(xb)          # 1. Generate predictions
      loss = loss_fn(pred, yb)  # 2. Calculate loss
      loss.backward()           # 3. Compute gradients
      opt.step()                # 4. Update parameters using gradients
      opt.zero_grad()           # 5. Reset the gradients to zero
    if (epoch+1) % nStep == 0:# Print the progress
      print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

fit(nEpoch, model, loss_fn, opt, train_dl)# train the model
w=model.weight.detach().numpy()[0]
b0=model.bias.detach().numpy()[0]
pred1=model(torch.tensor([point_of_interest]))
#print('\nPrediction for',point_of_interest,' is',float(pred1[0][0]) )
s='\nCurrent Model: '+dependant+" = "+str(np.round(b0,m))
i=0
for col in X.columns:
    s+=' + ('+str(np.round(w[i],m))+') * '+col
    i+=1
print(s)
print('\nPrediction for or X =', point_of_interest,'is  Y =',b0+(w*point_of_interest).sum())

   Sales (Y)  Calls (X1)  Time (X2)  Years (X3)   Type
0         37         131       16.9           2  GROUP
1         44         134       19.4           3  GROUP
2         33         139       19.3           3  GROUP
3         40         145       14.7           2  GROUP
4         35         149       15.6           1  GROUP
Epoch [10/100], Loss: 33.9290
Epoch [20/100], Loss: 204.0234
Epoch [30/100], Loss: 36.2261
Epoch [40/100], Loss: 37.3209
Epoch [50/100], Loss: 170.5785
Epoch [60/100], Loss: 102.8801
Epoch [70/100], Loss: 58.0209
Epoch [80/100], Loss: 159.0791
Epoch [90/100], Loss: 157.5058
Epoch [100/100], Loss: 210.9425

Current Model: Sales (Y) = 0.043121 + (0.248415) * Calls (X1) + (0.23029) * Time (X2) + (-0.333607) * Years (X3)

Prediction for or X = [160, 17, 3.0] is  Y = 42.70359779894352


# 3: Predict with the tree classifier

In [ ]:
# https://www.youtube.com/watch?v=cKxRvEZd3Mw
from sklearn import tree
bumpy=1
smooth=0
##  Enter your data #################
fruit_of_interest=[125,bumpy]
#########################
features=[[150,bumpy],[170,bumpy],[140,smooth],[130,smooth]]
labels=['orange','orange','apple','apple']
clf=tree.DecisionTreeClassifier()
clf=clf.fit(features,labels)
print(str(fruit_of_interest)+' is '+str(clf.predict([fruit_of_interest])))

[125, 1] is ['apple']


# 4: Predict with the SVM classifier

In [ ]:
# https://www.youtube.com/watch?v=cKxRvEZd3Mw
from sklearn import svm
bumpy=1
smooth=0
##  Enter your data #################
fruit_of_interest=[125,bumpy]
#########################
features=[[150,bumpy],[170,bumpy],[140,smooth],[130,smooth]]
labels=['orange','orange','apple','apple']
clf=svm.SVC(gamma=0.001, C=100.)
clf=clf.fit(features,labels)
print(str(fruit_of_interest)+' is '+str(clf.predict([fruit_of_interest])))

[125, 1] is ['apple']
